In [2]:
import pandas as pd
import os

# 1. train / test / val 정보가 담긴 csv 불러오기
df_split = pd.read_csv("faceforensic++_train_test_val_split.csv")
# 예: 
#   id  train_type
# 0  0        test
# 1  1       train
# 2  2       train
# 3  3        test
# 4  4         val
# ...

# 2. ff++_c23.csv 불러오기
#   컬럼: [id, video_path, frame_path_folder, label, type] (이미 존재)
df_ff = pd.read_csv("ff++_c23.csv")

# 3. 동영상 파일명에서 ID를 뽑아내서 train/test/val을 매칭해 주는 함수
def get_train_type_from_filename(filename: str) -> str:
    """
    파일명에서 ID를 추출 후, faceforensic++_train_test_val_split.csv의 train_type을 반환
    예) '000_003.mp4' -> '000'
        '998.mp4' -> '998'
    """
    # 확장자 제거 (.mp4 등)
    name_only = os.path.splitext(filename)[0]  # 예: '000_003' 또는 '998'
    
    # '_' 기준으로 앞부분이 우리가 원하는 ID(Original인 경우 '_' 자체가 없으므로 그대로 쓰면 됨)
    if '_' in name_only:
        name_only = name_only.split('_')[0]  # 예: '000_003' → '000'
    
    # 정수 변환 (문자열 → 숫자)
    id_num = int(name_only)
    
    # df_split에서 해당 ID에 맞는 train_type 가져오기
    row = df_split[df_split["id"] == id_num]
    
    # 혹시 해당 ID가 없을 경우 대비
    if len(row) == 0:
        return "Unknown"
    
    return row["train_type"].values[0]  # train / test / val

# 4. ff++_c23.csv의 각 행에 대해 video_path에서 파일명을 추출한 뒤 위 함수로 type 컬럼 채우기
def fill_type_column(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["type"] = df["video_path"].apply(lambda path: 
                                        get_train_type_from_filename(os.path.basename(path)))
    return df

# 실제로 type 컬럼 갱신
df_ff_updated = fill_type_column(df_ff)

# 5. 결과 확인
print(df_ff_updated.head())

# 6. 저장(필요하면)
df_ff_updated.to_csv("ff++_c23_with_type.csv", index=False)


          id                                         video_path  \
0  Deepfakes  /media/NAS/DATASET/FaceForensics_Dec2020/FaceF...   
1  Deepfakes  /media/NAS/DATASET/FaceForensics_Dec2020/FaceF...   
2  Deepfakes  /media/NAS/DATASET/FaceForensics_Dec2020/FaceF...   
3  Deepfakes  /media/NAS/DATASET/FaceForensics_Dec2020/FaceF...   
4  Deepfakes  /media/NAS/DATASET/FaceForensics_Dec2020/FaceF...   

                                   frame_path_folder  label   type  
0  /media/NAS/DATASET/faceforensics++/Dec2020/v1f...      1   test  
1  /media/NAS/DATASET/faceforensics++/Dec2020/v1f...      1  train  
2  /media/NAS/DATASET/faceforensics++/Dec2020/v1f...      1  train  
3  /media/NAS/DATASET/faceforensics++/Dec2020/v1f...      1   test  
4  /media/NAS/DATASET/faceforensics++/Dec2020/v1f...      1    val  
